#When running this file, first run the **Data preprocessing for input** Jupyter notebook located in the `preprocessing_structure_feature` folder.

In [13]:
import os
import sys
import logging
import umap
import numpy as np
import pandas as pd
import argparse
from sklearn import metrics
from datetime import datetime

path='./maincode/'
if path not in sys.path:
    sys.path.append(path)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from transformers import TFBertModel, BertTokenizer

from config import DeepAIR_BRP_saved_model_dict
from deepair.modelling.classification import SeqClassificationModelWithProcessor
from deepair.utility.utility import df_to_input_feature_extraction
from deepair.utility.utility import generate_AF2_feature_on_the_fly

from DeepAIR_Transformer_Feature_Extraction import generate_transformer_feature_on_the_fly

#%%
def model_seed_set(model_seed=13):
    os.environ['PYTHONHASHSEED']=str(model_seed)
    np.random.seed(model_seed)
    tf.random.set_seed(model_seed)
    
def map_probability_to_prediction(input, threshold):
    output = list()
    N, M = np.shape(input)
    for i in range(N):
        output.append(int(input[i,0]>=threshold))
    return output    

def calculate_performance(y_test, preds, output_folder, curr_epitope, output_per_note='AUC'):
    output_performance = {output_per_note: metrics.roc_auc_score(y_test, preds)}
    output_performance_value = output_performance[output_per_note]
    print(" \n\n model evaluation : ", output_performance)
    print(" \n\n ")
    output_file = os.path.join(output_folder, curr_epitope+'_Test{}-{}_performance.txt'.format(output_per_note, output_performance_value))
    with open(output_file, 'w+') as f:
        print(" \n\n model evaluation : ", output_performance, file=f)

def predict(input_data_file, 
            transformer_model_folder,
            seq_transformer_info,
            AF2_Feature_Info, 
            selected_epitope = None, 
            output_folder = None,
            label_column_name = None,
            output_per_note='AUC',
            task_name='unseen'
            ):
    
    # print('-'*30)
    # print(f'current epitope is {epitope}')
    
    if output_folder:
        os.makedirs(output_folder, exist_ok=True)
    
    df = pd.read_csv(input_data_file)  

    if 'Heavy_cdr3' in list(df.columns):
        cell_type = 'BCELL'
        input_df=df.rename(columns={'Heavy_cdr3':'TRB_cdr3',
                            'Heavy_v_gene':'TRB_v_gene',
                            'Heavy_j_gene':'TRB_j_gene',
                            'Light_cdr3':'TRA_cdr3',
                            'Light_v_gene':'TRA_v_gene',
                            'Light_j_gene':'TRA_j_gene',
                            })
    else:
        cell_type = 'TCELL'
        input_df=df
        
    # add transformer features
    seq_trans_beta_feature, seq_trans_alpha_feature = generate_transformer_feature_on_the_fly(input_df,
                                                         transformer_model_folder,
                                                         seq_transformer_info = seq_transformer_info
        )    
    # add AF2 structure features
    AF2_feature_beta_feature, AF2_feature_alpha_feature = generate_AF2_feature_on_the_fly(input_df, 
                                                                                        AF2_Feature_Info,
                                                                                        CDR3b = 'TRB_cdr3', 
                                                                                        b_vgene = 'TRB_v_gene',
                                                                                        b_jgene = 'TRB_j_gene',
                                                                                        CDR3a = 'TRA_cdr3', 
                                                                                        a_vgene = 'TRA_v_gene',
                                                                                        a_jgene = 'TRA_j_gene',
                                                                                        task_name = task_name,
                                                                                        ID = 'ID', 
                                                                                    )

    test_input = df_to_input_feature_extraction(input_df)
    test_input['TRB_cdr3_splited'] = seq_trans_beta_feature
    test_input['TRA_cdr3_splited'] = seq_trans_alpha_feature
    test_input['TRB_cdr3_Stru'] = AF2_feature_beta_feature
    test_input['TRA_cdr3_Stru'] = AF2_feature_alpha_feature
        
    output_df = pd.DataFrame()
    if cell_type == 'TCELL':
        output_df['ID'] = test_input['ID']
        output_df['TRB_cdr3'] = test_input['TRB_cdr3']
        output_df['TRA_cdr3'] = test_input['TRA_cdr3']
        output_df['TRB_v_gene'] = test_input['TRB_v_gene']
        output_df['TRB_j_gene'] = test_input['TRB_j_gene']
        output_df['TRA_v_gene'] = test_input['TRA_v_gene']
        output_df['TRA_j_gene'] = test_input['TRA_j_gene']
    else:
        output_df['ID'] = test_input['ID']
        output_df['Heavy_cdr3'] = test_input['TRB_cdr3']
        output_df['Light_cdr3'] = test_input['TRA_cdr3']
        output_df['Heavy_v_gene'] = test_input['TRB_v_gene']
        output_df['Heavy_j_gene'] = test_input['TRB_j_gene']
        output_df['Light_v_gene'] = test_input['TRA_v_gene']
        output_df['Light_j_gene'] = test_input['TRA_j_gene']
    
    for curr_epitope in selected_epitope: 
        
        
        model_save_path = DeepAIR_BRP_saved_model_dict[curr_epitope]
        
        # now load the saved model, from (tempoarary) file, into notebook
        loaded_model = SeqClassificationModelWithProcessor.from_file(model_save_path)
        # print('1.-'*30)
        # print(tf.executing_eagerly())               
    
        # # check the model ROC is the same as before
        preds = loaded_model.run(test_input)
        if tf.is_tensor(preds):
            preds = preds.numpy()
        
        # threshold = DeepAIR_BRP_cutoff_point_dict[curr_epitope]
        # y_test = map_probability_to_prediction(preds, threshold)
        
        output_df[curr_epitope+'_prob'] = preds
        if not label_column_name:
            if 'labels' in list(input_df.columns):
                output_df['Label'] = input_df['labels']
                y_test = input_df['labels'].to_numpy()
                calculate_performance(y_test, preds, output_folder, curr_epitope, output_per_note)
        else:
            output_df['Label'] = input_df[label_column_name]
            y_test = input_df[label_column_name].to_numpy()
            calculate_performance(y_test, preds, output_folder, curr_epitope, output_per_note)

    output_df.to_csv(os.path.join(output_folder,'prediction_results.csv'))
    
    return output_df



In [15]:
def fix(data_ori):
    data=pd.read_csv(data_ori)
    #'CDR3.beta', 'antigen_epitope','mhc.a','label','negative.source','license'
    data.rename(columns={'CDR3A':'TRA_cdr3','CDR3B':'TRB_cdr3','TRAV':'TRA_v_gene','TRBV':'TRB_v_gene', 'TRAJ':'TRA_j_gene','TRBJ':'TRB_j_gene', 
                         'LongA':'a_aaseq', 'LongB':'b_aaseq','Affinity':'y_true'},inplace=True)
    
    data['ID']=list(range(len(data)))

    df=data[['ID','TRA_cdr3','TRB_cdr3','TRA_v_gene','TRB_v_gene','TRA_j_gene','TRB_j_gene','a_aaseq','b_aaseq','Epitope','y_true']]
    
    return df


def Model_retraining(epitope,data_path,AF2_feature_folder):
    # epitope='A0301_KLGGALQAK_IE-1_CMV', #'Select an interested epitope'
    input_data_file=data_path#
    result_folder=f'./results/{epitope}'#
    os.makedirs(result_folder, exist_ok=True)
    
    # AF2_feature_folder=f'./preprocessing_structure_feature/out_folder_step3/'
    transformer_model_folder='./ProtTrans/prot_bert_bfd'
    
    mode='combined'#help='seq_only, stru_only, combined'
    predictor_info='GatedFusion'#help='CNN, GatedFusion'
    SeqEncoderTrans=True#help='transformer encoder'
    AF2_Info='Feature' #help='3D_structure, Feature'
    label_column_name=None
    dataset_name='unseen'
    model_seed=42

    
    # setting network seeds
    model_seed_set(model_seed = model_seed)

    # seq_transformer_info
    if SeqEncoderTrans:
        #---------------------------------------------------#
        transformer_tokenizer_name = os.path.abspath(transformer_model_folder)
        transformer_model_name = os.path.abspath(transformer_model_folder)    
        transformer_tokenizer = BertTokenizer.from_pretrained(transformer_tokenizer_name, do_lower_case=False)
        SeqInforModel = TFBertModel.from_pretrained(transformer_model_name, from_pt=False)
        SeqInforModel.trainable = False
        #---------------------------------------------------#    
    else:
        transformer_tokenizer = None
        SeqInforModel = None
    
    seq_transformer_info = dict()
    seq_transformer_info['whether_use_transformer'] = SeqEncoderTrans
    seq_transformer_info['tokenizer'] = transformer_tokenizer
    seq_transformer_info['tokenizer_fea_len'] = 40
    seq_transformer_info['SeqInforModel'] = SeqInforModel
    seq_transformer_info['transformer_feature'] = 'pooler_output' # 'pooler_output', 'last_hidden_state'
    seq_transformer_info['Transformer_fea_len'] = 1024

    if AF2_Info == 'Feature':
        # AF2 Features
        AF2_Feature_Info = dict()
        AF2_Feature_Info['seq_max_len'] = 40
        AF2_Feature_Info['fea_dim'] = 384
        AF2_Feature_Info['feature_file'] = AF2_feature_folder
    else:
        AF2_Feature_Info = None
        
    if not isinstance (epitope, list):
        selected_epitope = [epitope]
    else:
        selected_epitope = epitope
        
    print('-'*30)
    print(selected_epitope)    
    output_value = predict(input_data_file, 
                            transformer_model_folder, 
                            seq_transformer_info, # encoder sequence
                            AF2_Feature_Info, # encoder structure
                            selected_epitope = selected_epitope,
                            output_folder= result_folder,
                            label_column_name = label_column_name,
                            task_name = dataset_name
                        ) #'TRB_v_gene','TRB_j_gene','TRA_v_gene','TRA_j_gene','TRB_cdr3','TRA_cdr3',

def aggregate_results(testfile,epi_list, savedir, result_path):
    y_pred=[];epi_ls=[]
    df_results=pd.DataFrame()
    for epitope in epi_list:
        data_path=f'{savedir}{epitope}.csv'
        df_epi=pd.read_csv(data_path)
        df_results=pd.concat([df_results,df_epi],axis=0)
        result_folder=f'./results/{epitope}/'
        df_pre=pd.read_csv(result_folder+'prediction_results.csv')
        y_pred.extend(df_pre[df_pre.columns[-1]].tolist())
        epi_ls.extend([epitope]*len(df_epi))
    df_results['Epi']=epi_ls
    df_tmp=pd.read_csv(testfile)
    for i in range(len(df_tmp)):
        df_results.loc[(df_results.TRB_cdr3==df_tmp['CDR3B'][i])&(df_results.TRA_cdr3==df_tmp['CDR3A'][i]),'y_true']=df_tmp['Affinity'][i]
    df_results['y_prob']=y_pred
    df_results['y_pred'] = df_results['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
    df_results.to_csv(result_path+'probability1.csv', index=False)
    print("Saving done")

In [ ]:
testfile_path="../../data/train_CDR3B_others.csv"
result_path="../../result_path/Retraining_model_prediction"
os.makedirs(result_path, exist_ok=True)
AF2_feature_folder= f'./preprocessing_structure_feature/out_folder_step3/' 
df_test=fix(testfile_path)
epi_list=df_test.Epitope.unique().tolist()
savedir = "../tmp_results/"
os.makedirs(savedir, exist_ok=True)
for epitope in epi_list:
    print('epitope:',epitope)
    df_epi=df_test.loc[df_test.Epitope==epitope]               
    data_path=f'{savedir}{epitope}.csv'
    df_epi.to_csv(data_path, index=False)
    Model_retraining(epitope,data_path, AF2_feature_folder)
aggregate_results(testfile_path, epi_list, savedir, result_path)
